# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [2]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432,
                        dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

### Zadanie 1

In [3]:

df = pd.read_sql("SELECT count(film_id), length FROM film GROUP BY length ORDER BY length", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/277259295.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT count(film_id), length FROM film GROUP BY length ORDER BY length", con=connection)


,count,length
0,5,46
1,7,47
2,11,48
3,5,49
4,9,50
...,...,...
135,10,181
136,6,182
137,5,183
138,8,184


### Zadanie 2

In [4]:
def ex_2(city):
    df = pd.read_sql("SELECT customer.first_name, customer.last_name FROM customer \
    INNER JOIN address ON customer.address_id = address.address_id \
    INNER JOIN city ON address.city_id = city.city_id \
    WHERE city = '{0}'".format(city), con=connection)
    return df
ex_2('London')

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/678401548.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT customer.first_name, customer.last_name FROM customer \


,first_name,last_name
0,Mattie,Hoffman
1,Cecil,Vines


### Zadanie 3

In [5]:
df = pd.read_sql("SELECT avg(amount) FROM payment", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/3786408876.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT avg(amount) FROM payment", con=connection)


,avg
0,4.200606


### Zadanie 4

In [6]:
df = pd.read_sql("SELECT count(film_id), category.name AS category FROM category \
    INNER JOIN film_category ON category.category_id = film_category.category_id \
    GROUP BY category.name", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/3364527922.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT count(film_id), category.name AS category FROM category \


,count,category
0,74,Sports
1,57,Classics
2,63,New
3,69,Family
4,58,Comedy
5,66,Animation
6,57,Travel
7,51,Music
8,62,Drama
9,56,Horror


### Zadanie 5

In [7]:
df = pd.read_sql("SELECT country, count(customer_id) FROM customer \
    INNER JOIN address ON customer.address_id = address.address_id \
    INNER JOIN city ON address.city_id = city.city_id \
    INNER JOIN country ON city.country_id = country.country_id \
    GROUP BY country ORDER BY country", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/159944132.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT country, count(customer_id) FROM customer \


,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


### Zadanie 6

In [8]:
df = pd.read_sql("SELECT count(customer.customer_id), store.store_id, manager_staff_id, \
    store.address_id, store.last_update FROM store \
    INNER JOIN staff ON staff.store_id = store.store_id \
    INNER JOIN rental ON rental.staff_id = staff.staff_id \
    INNER JOIN customer ON customer.customer_id = rental.customer_id \
    GROUP BY store.store_id HAVING count(customer.customer_id) BETWEEN 100 and 300", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/4208370585.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT count(customer.customer_id), store.store_id, manager_staff_id, \


,count,store_id,manager_staff_id,address_id,last_update


### Zadanie 7

In [9]:
df = pd.read_sql("SELECT customer.first_name, customer.last_name FROM film \
    INNER JOIN inventory ON film.film_id = inventory.film_id \
    INNER JOIN rental ON inventory.inventory_id = rental.inventory_id \
    INNER JOIN customer ON customer.customer_id = rental.customer_id \
    GROUP BY customer.customer_id having sum(film.length) > 200", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/58015772.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT customer.first_name, customer.last_name FROM film \


,first_name,last_name
0,Wanda,Patterson
1,Vivian,Ruiz
2,Dan,Paine
3,Priscilla,Lowe
4,Guy,Brownlee
...,...,...
594,Oscar,Aquino
595,Mitchell,Westmoreland
596,Judith,Cox
597,Doris,Reed


### Zadanie 8

In [10]:
df = pd.read_sql("SELECT avg(rental_rate) FROM film", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/2025164628.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT avg(rental_rate) FROM film", con=connection)


,avg
0,2.98


### Zadanie 9

In [11]:
df = pd.read_sql("SELECT avg(film.length), name FROM film \
    INNER JOIN film_category ON film.film_id = film_category.film_id \
    INNER JOIN category ON category.category_id = film_category.category_id\
    GROUP BY category.name \
    ORDER BY avg(film.length)", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/1225599696.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT avg(film.length), name FROM film \


,avg,name
0,108.196721,Sci-Fi
1,108.750000,Documentary
2,109.800000,Children
3,111.015152,Animation
4,111.126984,New
5,111.609375,Action
6,111.666667,Classics
7,112.482143,Horror
8,113.315789,Travel
9,113.647059,Music


### Zadanie 10

In [12]:
df = pd.read_sql("SELECT film.title, category.name, film.length FROM film \
    INNER JOIN film_category ON film.film_id = film_category.film_id \
    INNER JOIN category ON category.category_id = film_category.category_id\
    WHERE (category.name, film.length) in \
        (SELECT category.name, max(film.length) FROM film \
        INNER JOIN film_category ON film.film_id = film_category.film_id \
        INNER JOIN category ON category.category_id = film_category.category_id \
        GROUP BY category.name) \
    ORDER BY film.length", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/2393033883.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT film.title, category.name, film.length FROM film \


,title,name,length
0,Fury Murder,Children,178
1,Wrong Behavior,Children,178
2,Analyze Hoosiers,Horror,181
3,Love Suicides,Horror,181
4,Jacket Frisco,Drama,181
5,Young Language,Documentary,183
6,Frontier Cabin,New,183
7,Wife Turn,Documentary,183
8,Crystal Breaking,Foreign,184
9,Smoochy Control,Sports,184


### Zadanie 11

In [13]:
df = pd.read_sql("SELECT film.title, film.length FROM film \
    WHERE film.length IN \
        (SELECT max(film.length) FROM film) \
    ORDER BY film.title", con=connection)
df

C:\Users\szyme\AppData\Local\Temp/ipykernel_17740/518166330.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT film.title, film.length FROM film \


,title,length
0,Chicago North,185
1,Control Anthem,185
2,Darn Forrester,185
3,Gangs Pride,185
4,Home Pity,185
5,Muscle Bright,185
6,Pond Seattle,185
7,Soldiers Evolution,185
8,Sweet Brotherhood,185
9,Worst Banger,185


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

### film_in_category

In [15]:
import main

main.film_in_category(1)

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT film.title, language.name AS languge, category.name AS category FROM category \


,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


### number_films_in_category

In [16]:
main.number_films_in_category(1)

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT category.name AS category, count(film.film_id) FROM category \


,category,count
0,Action,64


### number_film_by_length

In [17]:
main.number_film_by_length(46, 52)

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT film.length, count(film.film_id) FROM film \


,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
5,51,7
6,52,7


### client_from_city

In [18]:
main.client_from_city("London")

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT city.city, customer.first_name, customer.last_name FROM customer \


,city,first_name,last_name
0,London,Mattie,Hoffman
1,London,Cecil,Vines


### avg_amount_by_length

In [19]:
main.avg_amount_by_length(69)

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:137: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT film.length, avg(payment.amount) FROM film \


,length,avg
0,69,3.951165


### client_by_sum_length

In [20]:
main.client_by_sum_length(5000)

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:165: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT customer.first_name, customer.last_name, sum(film.length) FROM film \


,first_name,last_name,sum
0,Tammy,Sanders,5065
1,Eleanor,Hunt,5360
2,Karl,Seal,5388


### category_statistic_length

In [23]:
main.category_statistic_length("Action")

c:\Users\szyme\Documents\Studia\V semestr\AiBD\Szymon_Szewczyk_AiBD\lab2_join-SzewczykSzym\main.py:192: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT category.name AS category, avg(film.length), sum(film.length), min(film.length), max(film.length) FROM film \


,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
